# TPC-H with Iceberg Tables

TPC-H is a decision support benchmark that offers business-oriented ad hoc queries.
More information can be found [here](http://www.tpc.org/tpch)

The queries are originally in SQL format and here they are implemented using the pandas API.

By defaults runs use Bodo. Hence, data is distributed in chunks across processes.

**To scale and run your application with multiple nodes you can use [Bodo platform](https://platform.bodo.ai/account/login)**

The following code imports bodo and verifies that the IPyParallel cluster is set up correctly

In [ ]:
import os
if os.environ.get("BODO_PLATFORM_WORKSPACE_UUID",'NA') == 'NA':
    print("You are not on Bodo Platform, running ipyparallel.. ")
    import ipyparallel as ipp
    import psutil; n = min(psutil.cpu_count(logical=False), 8)
    rc = ipp.Cluster(engines='mpi', n=n).start_and_connect_sync(activate=True)

In [ ]:
%autopx

In [2]:
import bodo
import time
import numpy as np
import pandas as pd


%px:   0%|          | 0/144 [00:00<?, ?tasks/s]

<p> Generate <b>AWS_ACCESS_KEY_ID, AWS_SECRET_ACCESS_KEY and AWS_DEFAULT_REGION </b> of where data is presnt. Add them in the below code cell. </p>

Nessie Endpoint -  Go to your Dremio account,  click on Settings -> Project Settings -> Nessie Endpoint
Dremio Token - Click on your Username botton left -> Account Settings -> Personal Access Tokens -> Generate Token ->  copy the value generated. 

In [3]:
import json
import os

path_to_conn_creds = "/shared/anudeep_nbs/credentials.json"
with open(path_to_conn_creds) as f:
    creds = json.load(f)

os.environ["AWS_ACCESS_KEY_ID"] = creds["aws"]["aws_access_key_id"]
os.environ["AWS_SECRET_ACCESS_KEY"] = creds["aws"]["aws_secret_access_key"]
os.environ["AWS_DEFAULT_REGION"] = creds["aws"]["aws_default_region"]

Nessie_Endpoint = creds["iceberg"]["nessie_endpoint"]
Dremio_Token = creds["iceberg"]["token"]

In [3]:
iceberg_url = f"iceberg+{Nessie_Endpoint}?type=nessie&authentication.type=BEARER&authentication.token={Dremio_Token}"

<a id="loading_data"></a>
## Loading data

In this section, we load the data required by the queries in pandas DataFrame.

In read_sql_table  method change the first arguement with the table name. To write the tables to iceberg using EMR , https://github.com/Bodo-inc/examples-dev/blob/EMR_ICEBURG_TEST/emr_iceburg_examples/emr_iceburg_read_write.ipynb . 


In [4]:
@bodo.jit
def run_queries(iceberg_url):
    #tracing.start()

    print("#" * 128)
    print("Hello")
    print("#" * 128)

    # Load the data
    start_time = time.time()


    lineitem = pd.read_sql_table("SF1_LINEITEM_PQ_A", iceberg_url,"")

    orders = pd.read_sql_table("SF1_ORDERS_PQ",iceberg_url,"")

    customer = pd.read_sql_table("SF1_CUSTOMER_PQ",iceberg_url,"")
    

    nation = pd.read_sql_table("SF1_NATION_PQ",iceberg_url,"")

    region = pd.read_sql_table("SF1_REGION_PQ",iceberg_url,"")
    
    supplier = pd.read_sql_table("SF1_SUPPLIER_PQ",iceberg_url,"")
    
    part = pd.read_sql_table("SF1_PART_PQ",iceberg_url,"")
    
    partsupp = pd.read_sql_table("SF1_PARTSUPP_PQ",iceberg_url,"")

    t1 = time.time()
    
    q01(lineitem)
    q02(part, partsupp, supplier, nation, region)
    q03(lineitem, orders, customer)
    q04(lineitem, orders)
    q05(lineitem, orders, customer, nation, region, supplier)
    q06(lineitem)
    q07(lineitem, supplier, orders, customer, nation)
    q08(part, lineitem, supplier, orders, customer, nation, region)
    q09(lineitem, orders, part, nation, partsupp, supplier)
    q10(lineitem, orders, customer, nation)
    q11(partsupp, supplier, nation)
    q12(lineitem, orders)
    q13(customer, orders)
    q14(lineitem, part)
    q15(lineitem, supplier)
    q16(part, partsupp, supplier)
    q17(lineitem, part)
    q18(lineitem, orders, customer)
    q19(lineitem, part)
    q20(lineitem, part, nation, partsupp, supplier)
    q21(lineitem, orders, supplier, nation)
    q22(customer, orders)

    print("Total Query time (s): ", time.time() - t1)

    print("#"*128)
    print("total execution time : ", time.time() - start_time)

    #tracing.dump("/tmp/bodo_iceburg_tpch_07072022_parquet")



In [15]:
@bodo.jit
def q01(lineitem):
    t1 = time.time()
    date = pd.Timestamp("1998-09-02")
    lineitem_filtered = lineitem.loc[
                        :,
                        [
                            "L_QUANTITY",
                            "L_EXTENDEDPRICE",
                            "L_DISCOUNT",
                            "L_TAX",
                            "L_RETURNFLAG",
                            "L_LINESTATUS",
                            "L_SHIPDATE",
                            "L_ORDERKEY",
                        ],
                        ]
    sel = lineitem_filtered.L_SHIPDATE <= date
    lineitem_filtered = lineitem_filtered[sel]
    lineitem_filtered["AVG_QTY"] = lineitem_filtered.L_QUANTITY
    lineitem_filtered["AVG_PRICE"] = lineitem_filtered.L_EXTENDEDPRICE
    lineitem_filtered["DISC_PRICE"] = lineitem_filtered.L_EXTENDEDPRICE * (
            1 - lineitem_filtered.L_DISCOUNT
    )
    lineitem_filtered["CHARGE"] = (
            lineitem_filtered.L_EXTENDEDPRICE
            * (1 - lineitem_filtered.L_DISCOUNT)
            * (1 + lineitem_filtered.L_TAX)
    )
    gb = lineitem_filtered.groupby(["L_RETURNFLAG", "L_LINESTATUS"], as_index=False)[
        "L_QUANTITY",
        "L_EXTENDEDPRICE",
        "DISC_PRICE",
        "CHARGE",
        "AVG_QTY",
        "AVG_PRICE",
        "L_DISCOUNT",
        "L_ORDERKEY",
    ]
    total = gb.agg(
        {
            "L_QUANTITY": "sum",
            "L_EXTENDEDPRICE": "sum",
            "DISC_PRICE": "sum",
            "CHARGE": "sum",
            "AVG_QTY": "mean",
            "AVG_PRICE": "mean",
            "L_DISCOUNT": "mean",
            "L_ORDERKEY": "count",
        }
    )
    total = total.sort_values(["L_RETURNFLAG", "L_LINESTATUS"])
    print(total.head())
    print("Q01 Execution time (s): ", time.time() - t1)


@bodo.jit
def q02(part, partsupp, supplier, nation, region):
    t1 = time.time()
    nation_filtered = nation.loc[:, ["N_NATIONKEY", "N_NAME", "N_REGIONKEY"]]
    region_filtered = region[(region["R_NAME"] == "EUROPE")]
    region_filtered = region_filtered.loc[:, ["R_REGIONKEY"]]
    r_n_merged = nation_filtered.merge(
        region_filtered, left_on="N_REGIONKEY", right_on="R_REGIONKEY", how="inner"
    )
    r_n_merged = r_n_merged.loc[:, ["N_NATIONKEY", "N_NAME"]]
    supplier_filtered = supplier.loc[
                        :,
                        [
                            "S_SUPPKEY",
                            "S_NAME",
                            "S_ADDRESS",
                            "S_NATIONKEY",
                            "S_PHONE",
                            "S_ACCTBAL",
                            "S_COMMENT",
                        ],
                        ]
    s_r_n_merged = r_n_merged.merge(
        supplier_filtered, left_on="N_NATIONKEY", right_on="S_NATIONKEY", how="inner"
    )
    s_r_n_merged = s_r_n_merged.loc[
                   :,
                   [
                       "N_NAME",
                       "S_SUPPKEY",
                       "S_NAME",
                       "S_ADDRESS",
                       "S_PHONE",
                       "S_ACCTBAL",
                       "S_COMMENT",
                   ],
                   ]
    partsupp_filtered = partsupp.loc[:, ["PS_PARTKEY", "PS_SUPPKEY", "PS_SUPPLYCOST"]]
    ps_s_r_n_merged = s_r_n_merged.merge(
        partsupp_filtered, left_on="S_SUPPKEY", right_on="PS_SUPPKEY", how="inner"
    )
    ps_s_r_n_merged = ps_s_r_n_merged.loc[
                      :,
                      [
                          "N_NAME",
                          "S_NAME",
                          "S_ADDRESS",
                          "S_PHONE",
                          "S_ACCTBAL",
                          "S_COMMENT",
                          "PS_PARTKEY",
                          "PS_SUPPLYCOST",
                      ],
                      ]
    part_filtered = part.loc[:, ["P_PARTKEY", "P_MFGR", "P_SIZE", "P_TYPE"]]
    part_filtered = part_filtered[
        (part_filtered["P_SIZE"] == 15)
        & (part_filtered["P_TYPE"].str.endswith("BRASS"))
        ]
    part_filtered = part_filtered.loc[:, ["P_PARTKEY", "P_MFGR"]]
    merged_df = part_filtered.merge(
        ps_s_r_n_merged, left_on="P_PARTKEY", right_on="PS_PARTKEY", how="inner"
    )
    merged_df = merged_df.loc[
                :,
                [
                    "N_NAME",
                    "S_NAME",
                    "S_ADDRESS",
                    "S_PHONE",
                    "S_ACCTBAL",
                    "S_COMMENT",
                    "PS_SUPPLYCOST",
                    "P_PARTKEY",
                    "P_MFGR",
                ],
                ]
    min_values = merged_df.groupby("P_PARTKEY", as_index=False)["PS_SUPPLYCOST"].min()
    min_values.columns = ["P_PARTKEY_CPY", "MIN_SUPPLYCOST"]
    merged_df = merged_df.merge(
        min_values,
        left_on=["P_PARTKEY", "PS_SUPPLYCOST"],
        right_on=["P_PARTKEY_CPY", "MIN_SUPPLYCOST"],
        how="inner",
    )
    total = merged_df.loc[
            :,
            [
                "S_ACCTBAL",
                "S_NAME",
                "N_NAME",
                "P_PARTKEY",
                "P_MFGR",
                "S_ADDRESS",
                "S_PHONE",
                "S_COMMENT",
            ],
            ]
    total = total.sort_values(
        by=["S_ACCTBAL", "N_NAME", "S_NAME", "P_PARTKEY"],
        ascending=[False, True, True, True],
    )
    print(total.head())
    print("Q02 Execution time (s): ", time.time() - t1)


@bodo.jit
def q03(lineitem, orders, customer):
    t1 = time.time()
    date = pd.Timestamp("1995-03-04")
    lineitem_filtered = lineitem.loc[
                        :, ["L_ORDERKEY", "L_EXTENDEDPRICE", "L_DISCOUNT", "L_SHIPDATE"]
                        ]
    orders_filtered = orders.loc[
                      :, ["O_ORDERKEY", "O_CUSTKEY", "O_ORDERDATE", "O_SHIPPRIORITY"]
                      ]
    customer_filtered = customer.loc[:, ["C_MKTSEGMENT", "C_CUSTKEY"]]
    lsel = lineitem_filtered.L_SHIPDATE > date
    osel = orders_filtered.O_ORDERDATE < date
    csel = customer_filtered.C_MKTSEGMENT == "HOUSEHOLD"
    flineitem = lineitem_filtered[lsel]
    forders = orders_filtered[osel]
    fcustomer = customer_filtered[csel]
    jn1 = fcustomer.merge(forders, left_on="C_CUSTKEY", right_on="O_CUSTKEY")
    jn2 = jn1.merge(flineitem, left_on="O_ORDERKEY", right_on="L_ORDERKEY")
    jn2["TMP"] = jn2.L_EXTENDEDPRICE * (1 - jn2.L_DISCOUNT)
    total = (
        jn2.groupby(["L_ORDERKEY", "O_ORDERDATE", "O_SHIPPRIORITY"], as_index=False)[
            "TMP"
        ]
        .sum()
        .sort_values(["TMP"], ascending=False)
    )
    res = total.loc[:, ["L_ORDERKEY", "TMP", "O_ORDERDATE", "O_SHIPPRIORITY"]]
    print(res.head(10))
    print("Q03 Execution time (s): ", time.time() - t1)


@bodo.jit
def q04(lineitem, orders):
    t1 = time.time()
    date1 = pd.Timestamp("1993-11-01")
    date2 = pd.Timestamp("1993-08-01")
    lsel = lineitem.L_COMMITDATE < lineitem.L_RECEIPTDATE
    osel = (orders.O_ORDERDATE < date1) & (orders.O_ORDERDATE >= date2)
    flineitem = lineitem[lsel]
    forders = orders[osel]
    jn = forders[forders["O_ORDERKEY"].isin(flineitem["L_ORDERKEY"])]
    total = (
        jn.groupby("O_ORDERPRIORITY", as_index=False)["O_ORDERKEY"]
        .count()
        .sort_values(["O_ORDERPRIORITY"])
    )
    print(total.head())
    print("Q04 Execution time (s): ", time.time() - t1)


@bodo.jit
def q05(lineitem, orders, customer, nation, region, supplier):
    t1 = time.time()
    date1 = pd.Timestamp("1996-01-01")
    date2 = pd.Timestamp("1997-01-01")
    rsel = region.R_NAME == "ASIA"
    osel = (orders.O_ORDERDATE >= date1) & (orders.O_ORDERDATE < date2)
    forders = orders[osel]
    fregion = region[rsel]
    jn1 = fregion.merge(nation, left_on="R_REGIONKEY", right_on="N_REGIONKEY")
    jn2 = jn1.merge(customer, left_on="N_NATIONKEY", right_on="C_NATIONKEY")
    jn3 = jn2.merge(forders, left_on="C_CUSTKEY", right_on="O_CUSTKEY")
    jn4 = jn3.merge(lineitem, left_on="O_ORDERKEY", right_on="L_ORDERKEY")
    jn5 = supplier.merge(
        jn4, left_on=["S_SUPPKEY", "S_NATIONKEY"], right_on=["L_SUPPKEY", "N_NATIONKEY"]
    )
    jn5["TMP"] = jn5.L_EXTENDEDPRICE * (1.0 - jn5.L_DISCOUNT)
    gb = jn5.groupby("N_NAME", as_index=False)["TMP"].sum()
    total = gb.sort_values("TMP", ascending=False)
    bodo.parallel_print(total.head(1))
    print("Q05 Execution time (s): ", time.time() - t1)


@bodo.jit
def q06(lineitem):
    t1 = time.time()
    date1 = pd.Timestamp("1996-01-01")
    date2 = pd.Timestamp("1997-01-01")
    lineitem_filtered = lineitem.loc[
                        :, ["L_QUANTITY", "L_EXTENDEDPRICE", "L_DISCOUNT", "L_SHIPDATE"]
                        ]
    sel = (
            (lineitem_filtered.L_SHIPDATE >= date1)
            & (lineitem_filtered.L_SHIPDATE < date2)
            & (lineitem_filtered.L_DISCOUNT >= 0.08)
            & (lineitem_filtered.L_DISCOUNT <= 0.1)
            & (lineitem_filtered.L_QUANTITY < 24)
    )
    flineitem = lineitem_filtered[sel]
    total = (flineitem.L_EXTENDEDPRICE * flineitem.L_DISCOUNT).sum()
    print(total)
    print("Q06 Execution time (s): ", time.time() - t1)


@bodo.jit
def q07(lineitem, supplier, orders, customer, nation):
    """This version is faster than q07_old. Keeping the old one for reference"""
    t1 = time.time()

    lineitem_filtered = lineitem[
        (lineitem["L_SHIPDATE"] >= pd.Timestamp("1995-01-01"))
        & (lineitem["L_SHIPDATE"] < pd.Timestamp("1997-01-01"))
        ]
    lineitem_filtered["L_YEAR"] = lineitem_filtered["L_SHIPDATE"].apply(
        lambda x: x.year
    )
    lineitem_filtered["VOLUME"] = lineitem_filtered["L_EXTENDEDPRICE"] * (
            1.0 - lineitem_filtered["L_DISCOUNT"]
    )
    lineitem_filtered = lineitem_filtered.loc[
                        :, ["L_ORDERKEY", "L_SUPPKEY", "L_YEAR", "VOLUME"]
                        ]
    supplier_filtered = supplier.loc[:, ["S_SUPPKEY", "S_NATIONKEY"]]
    orders_filtered = orders.loc[:, ["O_ORDERKEY", "O_CUSTKEY"]]
    customer_filtered = customer.loc[:, ["C_CUSTKEY", "C_NATIONKEY"]]
    n1 = nation[(nation["N_NAME"] == "FRANCE")].loc[:, ["N_NATIONKEY", "N_NAME"]]
    n2 = nation[(nation["N_NAME"] == "GERMANY")].loc[:, ["N_NATIONKEY", "N_NAME"]]

    # ----- do nation 1 -----
    N1_C = customer_filtered.merge(
        n1, left_on="C_NATIONKEY", right_on="N_NATIONKEY", how="inner"
    )
    N1_C = N1_C.drop(columns=["C_NATIONKEY", "N_NATIONKEY"]).rename(
        columns={"N_NAME": "CUST_NATION"}
    )
    N1_C_O = N1_C.merge(
        orders_filtered, left_on="C_CUSTKEY", right_on="O_CUSTKEY", how="inner"
    )
    N1_C_O = N1_C_O.drop(columns=["C_CUSTKEY", "O_CUSTKEY"])

    N2_S = supplier_filtered.merge(
        n2, left_on="S_NATIONKEY", right_on="N_NATIONKEY", how="inner"
    )
    N2_S = N2_S.drop(columns=["S_NATIONKEY", "N_NATIONKEY"]).rename(
        columns={"N_NAME": "SUPP_NATION"}
    )
    N2_S_L = N2_S.merge(
        lineitem_filtered, left_on="S_SUPPKEY", right_on="L_SUPPKEY", how="inner"
    )
    N2_S_L = N2_S_L.drop(columns=["S_SUPPKEY", "L_SUPPKEY"])

    total1 = N1_C_O.merge(
        N2_S_L, left_on="O_ORDERKEY", right_on="L_ORDERKEY", how="inner"
    )
    total1 = total1.drop(columns=["O_ORDERKEY", "L_ORDERKEY"])

    # ----- do nation 2 -----
    N2_C = customer_filtered.merge(
        n2, left_on="C_NATIONKEY", right_on="N_NATIONKEY", how="inner"
    )
    N2_C = N2_C.drop(columns=["C_NATIONKEY", "N_NATIONKEY"]).rename(
        columns={"N_NAME": "CUST_NATION"}
    )
    N2_C_O = N2_C.merge(
        orders_filtered, left_on="C_CUSTKEY", right_on="O_CUSTKEY", how="inner"
    )
    N2_C_O = N2_C_O.drop(columns=["C_CUSTKEY", "O_CUSTKEY"])

    N1_S = supplier_filtered.merge(
        n1, left_on="S_NATIONKEY", right_on="N_NATIONKEY", how="inner"
    )
    N1_S = N1_S.drop(columns=["S_NATIONKEY", "N_NATIONKEY"]).rename(
        columns={"N_NAME": "SUPP_NATION"}
    )
    N1_S_L = N1_S.merge(
        lineitem_filtered, left_on="S_SUPPKEY", right_on="L_SUPPKEY", how="inner"
    )
    N1_S_L = N1_S_L.drop(columns=["S_SUPPKEY", "L_SUPPKEY"])

    total2 = N2_C_O.merge(
        N1_S_L, left_on="O_ORDERKEY", right_on="L_ORDERKEY", how="inner"
    )
    total2 = total2.drop(columns=["O_ORDERKEY", "L_ORDERKEY"])

    # concat results
    total = pd.concat([total1, total2])

    total = total.groupby(["SUPP_NATION", "CUST_NATION", "L_YEAR"], as_index=False).agg(
        REVENUE=pd.NamedAgg(column="VOLUME", aggfunc="sum")
    )
    total = total.sort_values(
        by=["SUPP_NATION", "CUST_NATION", "L_YEAR"], ascending=[True, True, True]
    )
    print(total.head())
    print("Q07 Execution time (s): ", time.time() - t1)


@bodo.jit
def q08(part, lineitem, supplier, orders, customer, nation, region):
    t1 = time.time()
    part_filtered = part[(part["P_TYPE"] == "ECONOMY ANODIZED STEEL")]
    part_filtered = part_filtered.loc[:, ["P_PARTKEY"]]
    lineitem_filtered = lineitem.loc[:, ["L_PARTKEY", "L_SUPPKEY", "L_ORDERKEY"]]
    lineitem_filtered["VOLUME"] = lineitem["L_EXTENDEDPRICE"] * (
            1.0 - lineitem["L_DISCOUNT"]
    )
    total = part_filtered.merge(
        lineitem_filtered, left_on="P_PARTKEY", right_on="L_PARTKEY", how="inner"
    )
    total = total.loc[:, ["L_SUPPKEY", "L_ORDERKEY", "VOLUME"]]
    supplier_filtered = supplier.loc[:, ["S_SUPPKEY", "S_NATIONKEY"]]
    total = total.merge(
        supplier_filtered, left_on="L_SUPPKEY", right_on="S_SUPPKEY", how="inner"
    )
    total = total.loc[:, ["L_ORDERKEY", "VOLUME", "S_NATIONKEY"]]
    orders_filtered = orders[
        (orders["O_ORDERDATE"] >= pd.Timestamp("1995-01-01"))
        & (orders["O_ORDERDATE"] < pd.Timestamp("1997-01-01"))
        ]
    orders_filtered["O_YEAR"] = orders_filtered["O_ORDERDATE"].apply(lambda x: x.year)
    orders_filtered = orders_filtered.loc[:, ["O_ORDERKEY", "O_CUSTKEY", "O_YEAR"]]
    total = total.merge(
        orders_filtered, left_on="L_ORDERKEY", right_on="O_ORDERKEY", how="inner"
    )
    total = total.loc[:, ["VOLUME", "S_NATIONKEY", "O_CUSTKEY", "O_YEAR"]]
    customer_filtered = customer.loc[:, ["C_CUSTKEY", "C_NATIONKEY"]]
    total = total.merge(
        customer_filtered, left_on="O_CUSTKEY", right_on="C_CUSTKEY", how="inner"
    )
    total = total.loc[:, ["VOLUME", "S_NATIONKEY", "O_YEAR", "C_NATIONKEY"]]
    n1_filtered = nation.loc[:, ["N_NATIONKEY", "N_REGIONKEY"]]
    n2_filtered = nation.loc[:, ["N_NATIONKEY", "N_NAME"]].rename(
        columns={"N_NAME": "NATION"}
    )
    total = total.merge(
        n1_filtered, left_on="C_NATIONKEY", right_on="N_NATIONKEY", how="inner"
    )
    total = total.loc[:, ["VOLUME", "S_NATIONKEY", "O_YEAR", "N_REGIONKEY"]]
    total = total.merge(
        n2_filtered, left_on="S_NATIONKEY", right_on="N_NATIONKEY", how="inner"
    )
    total = total.loc[:, ["VOLUME", "O_YEAR", "N_REGIONKEY", "NATION"]]
    region_filtered = region[(region["R_NAME"] == "AMERICA")]
    region_filtered = region_filtered.loc[:, ["R_REGIONKEY"]]
    total = total.merge(
        region_filtered, left_on="N_REGIONKEY", right_on="R_REGIONKEY", how="inner"
    )
    total = total.loc[:, ["VOLUME", "O_YEAR", "NATION"]]

    def udf(df):
        demonimator = df["VOLUME"].sum()
        df = df[df["NATION"] == "BRAZIL"]
        numerator = df["VOLUME"].sum()
        return numerator / demonimator

    total = total.groupby("O_YEAR", as_index=False).apply(udf)
    total.columns = ["O_YEAR", "MKT_SHARE"]
    total = total.sort_values(by=["O_YEAR"], ascending=[True])
    print(total.head())
    print("Q08 Execution time (s): ", time.time() - t1)


@bodo.jit
def q09(lineitem, orders, part, nation, partsupp, supplier):
    t1 = time.time()
    psel = part.P_NAME.str.contains("ghost")
    fpart = part[psel]
    jn1 = lineitem.merge(fpart, left_on="L_PARTKEY", right_on="P_PARTKEY")
    jn2 = jn1.merge(supplier, left_on="L_SUPPKEY", right_on="S_SUPPKEY")
    jn3 = jn2.merge(nation, left_on="S_NATIONKEY", right_on="N_NATIONKEY")
    jn4 = partsupp.merge(
        jn3, left_on=["PS_PARTKEY", "PS_SUPPKEY"], right_on=["L_PARTKEY", "L_SUPPKEY"]
    )
    jn5 = jn4.merge(orders, left_on="L_ORDERKEY", right_on="O_ORDERKEY")
    jn5["TMP"] = jn5.L_EXTENDEDPRICE * (1 - jn5.L_DISCOUNT) - (
            (1 * jn5.PS_SUPPLYCOST) * jn5.L_QUANTITY
    )
    jn5["O_YEAR"] = jn5.O_ORDERDATE.apply(lambda x: x.year)
    gb = jn5.groupby(["N_NAME", "O_YEAR"], as_index=False)["TMP"].sum()
    total = gb.sort_values(["N_NAME", "O_YEAR"], ascending=[True, False])
    print(total.head())
    print("Q09 Execution time (s): ", time.time() - t1)


@bodo.jit
def q10(lineitem, orders, customer, nation):
    t1 = time.time()
    date1 = pd.Timestamp("1994-11-01")
    date2 = pd.Timestamp("1995-02-01")
    osel = (orders.O_ORDERDATE >= date1) & (orders.O_ORDERDATE < date2)
    lsel = lineitem.L_RETURNFLAG == "R"
    forders = orders[osel]
    flineitem = lineitem[lsel]
    jn1 = flineitem.merge(forders, left_on="L_ORDERKEY", right_on="O_ORDERKEY")
    jn2 = jn1.merge(customer, left_on="O_CUSTKEY", right_on="C_CUSTKEY")
    jn3 = jn2.merge(nation, left_on="C_NATIONKEY", right_on="N_NATIONKEY")
    jn3["TMP"] = jn3.L_EXTENDEDPRICE * (1.0 - jn3.L_DISCOUNT)
    gb = jn3.groupby(
        [
            "C_CUSTKEY",
            "C_NAME",
            "C_ACCTBAL",
            "C_PHONE",
            "N_NAME",
            "C_ADDRESS",
            "C_COMMENT",
        ],
        as_index=False,
    )["TMP"].sum()
    total = gb.sort_values("TMP", ascending=False)
    print(total.head(20))
    print("Q10 Execution time (s): ", time.time() - t1)


@bodo.jit
def q11(partsupp, supplier, nation):
    t1 = time.time()
    partsupp_filtered = partsupp.loc[:, ["PS_PARTKEY", "PS_SUPPKEY"]]
    partsupp_filtered["TOTAL_COST"] = (
            partsupp["PS_SUPPLYCOST"] * partsupp["PS_AVAILQTY"]
    )
    supplier_filtered = supplier.loc[:, ["S_SUPPKEY", "S_NATIONKEY"]]
    ps_supp_merge = partsupp_filtered.merge(
        supplier_filtered, left_on="PS_SUPPKEY", right_on="S_SUPPKEY", how="inner"
    )
    ps_supp_merge.loc[:, ["PS_PARTKEY", "S_NATIONKEY", "TOTAL_COST"]]
    nation_filtered = nation[(nation["N_NAME"] == "GERMANY")]
    nation_filtered = nation_filtered.loc[:, ["N_NATIONKEY"]]
    ps_supp_n_merge = ps_supp_merge.merge(
        nation_filtered, left_on="S_NATIONKEY", right_on="N_NATIONKEY", how="inner"
    )
    ps_supp_n_merge = ps_supp_n_merge.loc[:, ["PS_PARTKEY", "TOTAL_COST"]]
    sum_val = ps_supp_n_merge["TOTAL_COST"].sum() * 0.0001
    total = ps_supp_n_merge.groupby(["PS_PARTKEY"], as_index=False).agg(
        VALUE=pd.NamedAgg(column="TOTAL_COST", aggfunc="sum")
    )
    total = total[total["VALUE"] > sum_val]
    total = total.sort_values("VALUE", ascending=False)
    print(total)
    print("Q11 Execution time (s): ", time.time() - t1)


@bodo.jit
def q12(lineitem, orders):
    t1 = time.time()
    date1 = pd.Timestamp("1994-01-01")
    date2 = pd.Timestamp("1995-01-01")
    sel = (
            (lineitem.L_RECEIPTDATE < date2)
            & (lineitem.L_COMMITDATE < date2)
            & (lineitem.L_SHIPDATE < date2)
            & (lineitem.L_SHIPDATE < lineitem.L_COMMITDATE)
            & (lineitem.L_COMMITDATE < lineitem.L_RECEIPTDATE)
            & (lineitem.L_RECEIPTDATE >= date1)
            & ((lineitem.L_SHIPMODE == "MAIL") | (lineitem.L_SHIPMODE == "SHIP"))
    )
    flineitem = lineitem[sel]
    jn = flineitem.merge(orders, left_on="L_ORDERKEY", right_on="O_ORDERKEY")

    def g1(x):
        return ((x == "1-URGENT") | (x == "2-HIGH")).sum()

    def g2(x):
        return ((x != "1-URGENT") & (x != "2-HIGH")).sum()

    total = jn.groupby("L_SHIPMODE", as_index=False)["O_ORDERPRIORITY"].agg((g1, g2))
    total = total.sort_values("L_SHIPMODE")
    print(total.head())
    print("Q12 Execution time (s): ", time.time() - t1)


@bodo.jit
def q13(customer, orders):
    t1 = time.time()
    customer_filtered = customer.loc[:, ["C_CUSTKEY"]]
    orders_filtered = orders[
        ~orders["O_COMMENT"].str.contains("special(\S|\s)*requests")
    ]
    orders_filtered = orders_filtered.loc[:, ["O_ORDERKEY", "O_CUSTKEY"]]
    c_o_merged = customer_filtered.merge(
        orders_filtered, left_on="C_CUSTKEY", right_on="O_CUSTKEY", how="left"
    )
    c_o_merged = c_o_merged.loc[:, ["C_CUSTKEY", "O_ORDERKEY"]]
    count_df = c_o_merged.groupby(["C_CUSTKEY"], as_index=False).agg(
        C_COUNT=pd.NamedAgg(column="O_ORDERKEY", aggfunc="count")
    )
    total = count_df.groupby(["C_COUNT"], as_index=False).size()
    total.columns = ["C_COUNT", "CUSTDIST"]
    total = total.sort_values(by=["CUSTDIST", "C_COUNT"], ascending=[False, False])
    print(total.head())
    print("Q13 Execution time (s): ", time.time() - t1)


@bodo.jit
def q14(lineitem, part):
    t1 = time.time()
    startDate = pd.Timestamp("1994-03-01")
    endDate = pd.Timestamp("1994-04-01")
    p_type_like = "PROMO"
    part_filtered = part.loc[:, ["P_PARTKEY", "P_TYPE"]]
    lineitem_filtered = lineitem.loc[
                        :, ["L_EXTENDEDPRICE", "L_DISCOUNT", "L_SHIPDATE", "L_PARTKEY"]
                        ]
    sel = (lineitem_filtered.L_SHIPDATE >= startDate) & (
            lineitem_filtered.L_SHIPDATE < endDate
    )
    flineitem = lineitem_filtered[sel]
    jn = flineitem.merge(part_filtered, left_on="L_PARTKEY", right_on="P_PARTKEY")
    jn["TMP"] = jn.L_EXTENDEDPRICE * (1.0 - jn.L_DISCOUNT)
    total = jn[jn.P_TYPE.str.startswith(p_type_like)].TMP.sum() * 100 / jn.TMP.sum()
    print(total)
    print("Q14 Execution time (s): ", time.time() - t1)


@bodo.jit
def q15(lineitem, supplier):
    t1 = time.time()
    lineitem_filtered = lineitem[
        (lineitem["L_SHIPDATE"] >= pd.Timestamp("1996-01-01"))
        & (
                lineitem["L_SHIPDATE"]
                < (pd.Timestamp("1996-01-01") + pd.DateOffset(months=3))
        )
        ]
    lineitem_filtered["REVENUE_PARTS"] = lineitem_filtered["L_EXTENDEDPRICE"] * (
            1.0 - lineitem_filtered["L_DISCOUNT"]
    )
    lineitem_filtered = lineitem_filtered.loc[:, ["L_SUPPKEY", "REVENUE_PARTS"]]
    revenue_table = (
        lineitem_filtered.groupby("L_SUPPKEY", as_index=False)
        .agg(TOTAL_REVENUE=pd.NamedAgg(column="REVENUE_PARTS", aggfunc="sum"))
        .rename(columns={"L_SUPPKEY": "SUPPLIER_NO"}, copy=False)
    )
    max_revenue = revenue_table["TOTAL_REVENUE"].max()
    revenue_table = revenue_table[revenue_table["TOTAL_REVENUE"] == max_revenue]
    supplier_filtered = supplier.loc[:, ["S_SUPPKEY", "S_NAME", "S_ADDRESS", "S_PHONE"]]
    total = supplier_filtered.merge(
        revenue_table, left_on="S_SUPPKEY", right_on="SUPPLIER_NO", how="inner"
    )
    total = total.loc[
            :, ["S_SUPPKEY", "S_NAME", "S_ADDRESS", "S_PHONE", "TOTAL_REVENUE"]
            ]
    print(total.head())
    print("Q15 Execution time (s): ", time.time() - t1)


@bodo.jit
def q16(part, partsupp, supplier):
    t1 = time.time()
    part_filtered = part[
        (part["P_BRAND"] != "Brand#45")
        & (~part["P_TYPE"].str.contains("^MEDIUM POLISHED"))
        & part["P_SIZE"].isin([49, 14, 23, 45, 19, 3, 36, 9])
        ]
    part_filtered = part_filtered.loc[:, ["P_PARTKEY", "P_BRAND", "P_TYPE", "P_SIZE"]]
    partsupp_filtered = partsupp.loc[:, ["PS_PARTKEY", "PS_SUPPKEY"]]
    total = part_filtered.merge(
        partsupp_filtered, left_on="P_PARTKEY", right_on="PS_PARTKEY", how="inner"
    )
    total = total.loc[:, ["P_BRAND", "P_TYPE", "P_SIZE", "PS_SUPPKEY"]]
    supplier_filtered = supplier[
        supplier["S_COMMENT"].str.contains("Customer(\S|\s)*Complaints")
    ]
    supplier_filtered = supplier_filtered.loc[:, ["S_SUPPKEY"]].drop_duplicates()
    # left merge to select only PS_SUPPKEY values not in supplier_filtered
    total = total.merge(
        supplier_filtered, left_on="PS_SUPPKEY", right_on="S_SUPPKEY", how="left"
    )
    total = total[total["S_SUPPKEY"].isna()]
    total = total.loc[:, ["P_BRAND", "P_TYPE", "P_SIZE", "PS_SUPPKEY"]]
    total = total.groupby(["P_BRAND", "P_TYPE", "P_SIZE"], as_index=False)[
        "PS_SUPPKEY"
    ].nunique()
    total.columns = ["P_BRAND", "P_TYPE", "P_SIZE", "SUPPLIER_CNT"]
    total = total.sort_values(
        by=["SUPPLIER_CNT", "P_BRAND", "P_TYPE", "P_SIZE"],
        ascending=[False, True, True, True],
    )
    print(total.head())
    print("Q16 Execution time (s): ", time.time() - t1)


@bodo.jit
def q17(lineitem, part):
    t1 = time.time()
    left = lineitem.loc[:, ["L_PARTKEY", "L_QUANTITY", "L_EXTENDEDPRICE"]]
    right = part[((part["P_BRAND"] == "Brand#23") & (part["P_CONTAINER"] == "MED BOX"))]
    right = right.loc[:, ["P_PARTKEY"]]
    line_part_merge = left.merge(
        right, left_on="L_PARTKEY", right_on="P_PARTKEY", how="inner"
    )
    line_part_merge = line_part_merge.loc[
                      :, ["L_QUANTITY", "L_EXTENDEDPRICE", "P_PARTKEY"]
                      ]
    lineitem_filtered = lineitem.loc[:, ["L_PARTKEY", "L_QUANTITY"]]
    lineitem_avg = lineitem_filtered.groupby(["L_PARTKEY"], as_index=False).agg(
        avg=pd.NamedAgg(column="L_QUANTITY", aggfunc="mean")
    )
    lineitem_avg["avg"] = 0.2 * lineitem_avg["avg"]
    lineitem_avg = lineitem_avg.loc[:, ["L_PARTKEY", "avg"]]
    total = line_part_merge.merge(
        lineitem_avg, left_on="P_PARTKEY", right_on="L_PARTKEY", how="inner"
    )
    total = total[total["L_QUANTITY"] < total["avg"]]
    total = pd.DataFrame({"avg_yearly": [total["L_EXTENDEDPRICE"].sum() / 7.0]})
    print(total.head())
    print("Q17 Execution time (s): ", time.time() - t1)


@bodo.jit
def q18(lineitem, orders, customer):
    t1 = time.time()
    gb1 = lineitem.groupby("L_ORDERKEY", as_index=False)["L_QUANTITY"].sum()
    fgb1 = gb1[gb1.L_QUANTITY > 300]
    jn1 = fgb1.merge(orders, left_on="L_ORDERKEY", right_on="O_ORDERKEY")
    jn2 = jn1.merge(customer, left_on="O_CUSTKEY", right_on="C_CUSTKEY")
    gb2 = jn2.groupby(
        ["C_NAME", "C_CUSTKEY", "O_ORDERKEY", "O_ORDERDATE", "O_TOTALPRICE"],
        as_index=False,
    )["L_QUANTITY"].sum()
    total = gb2.sort_values(["O_TOTALPRICE", "O_ORDERDATE"], ascending=[False, True])
    print(total.head(10))
    print("Q18 Execution time (s): ", time.time() - t1)


@bodo.jit
def q19(lineitem, part):
    t1 = time.time()
    Brand31 = "Brand#31"
    Brand43 = "Brand#43"
    SMBOX = "SM BOX"
    SMCASE = "SM CASE"
    SMPACK = "SM PACK"
    SMPKG = "SM PKG"
    MEDBAG = "MED BAG"
    MEDBOX = "MED BOX"
    MEDPACK = "MED PACK"
    MEDPKG = "MED PKG"
    LGBOX = "LG BOX"
    LGCASE = "LG CASE"
    LGPACK = "LG PACK"
    LGPKG = "LG PKG"
    DELIVERINPERSON = "DELIVER IN PERSON"
    AIR = "AIR"
    AIRREG = "AIRREG"
    lsel = (
            (
                    ((lineitem.L_QUANTITY <= 36) & (lineitem.L_QUANTITY >= 26))
                    | ((lineitem.L_QUANTITY <= 25) & (lineitem.L_QUANTITY >= 15))
                    | ((lineitem.L_QUANTITY <= 14) & (lineitem.L_QUANTITY >= 4))
            )
            & (lineitem.L_SHIPINSTRUCT == DELIVERINPERSON)
            & ((lineitem.L_SHIPMODE == AIR) | (lineitem.L_SHIPMODE == AIRREG))
    )
    psel = (part.P_SIZE >= 1) & (
            (
                    (part.P_SIZE <= 5)
                    & (part.P_BRAND == Brand31)
                    & (
                            (part.P_CONTAINER == SMBOX)
                            | (part.P_CONTAINER == SMCASE)
                            | (part.P_CONTAINER == SMPACK)
                            | (part.P_CONTAINER == SMPKG)
                    )
            )
            | (
                    (part.P_SIZE <= 10)
                    & (part.P_BRAND == Brand43)
                    & (
                            (part.P_CONTAINER == MEDBAG)
                            | (part.P_CONTAINER == MEDBOX)
                            | (part.P_CONTAINER == MEDPACK)
                            | (part.P_CONTAINER == MEDPKG)
                    )
            )
            | (
                    (part.P_SIZE <= 15)
                    & (part.P_BRAND == Brand43)
                    & (
                            (part.P_CONTAINER == LGBOX)
                            | (part.P_CONTAINER == LGCASE)
                            | (part.P_CONTAINER == LGPACK)
                            | (part.P_CONTAINER == LGPKG)
                    )
            )
    )
    flineitem = lineitem[lsel]
    fpart = part[psel]
    jn = flineitem.merge(fpart, left_on="L_PARTKEY", right_on="P_PARTKEY")
    jnsel = (
            (jn.P_BRAND == Brand31)
            & (
                    (jn.P_CONTAINER == SMBOX)
                    | (jn.P_CONTAINER == SMCASE)
                    | (jn.P_CONTAINER == SMPACK)
                    | (jn.P_CONTAINER == SMPKG)
            )
            & (jn.L_QUANTITY >= 4)
            & (jn.L_QUANTITY <= 14)
            & (jn.P_SIZE <= 5)
            | (jn.P_BRAND == Brand43)
            & (
                    (jn.P_CONTAINER == MEDBAG)
                    | (jn.P_CONTAINER == MEDBOX)
                    | (jn.P_CONTAINER == MEDPACK)
                    | (jn.P_CONTAINER == MEDPKG)
            )
            & (jn.L_QUANTITY >= 15)
            & (jn.L_QUANTITY <= 25)
            & (jn.P_SIZE <= 10)
            | (jn.P_BRAND == Brand43)
            & (
                    (jn.P_CONTAINER == LGBOX)
                    | (jn.P_CONTAINER == LGCASE)
                    | (jn.P_CONTAINER == LGPACK)
                    | (jn.P_CONTAINER == LGPKG)
            )
            & (jn.L_QUANTITY >= 26)
            & (jn.L_QUANTITY <= 36)
            & (jn.P_SIZE <= 15)
    )
    jn = jn[jnsel]
    total = (jn.L_EXTENDEDPRICE * (1.0 - jn.L_DISCOUNT)).sum()
    print(total)
    print("Q19 Execution time (s): ", time.time() - t1)


@bodo.jit
def q20(lineitem, part, nation, partsupp, supplier):
    t1 = time.time()
    date1 = pd.Timestamp("1996-01-01")
    date2 = pd.Timestamp("1997-01-01")
    psel = part.P_NAME.str.startswith("azure")
    nsel = nation.N_NAME == "JORDAN"
    lsel = (lineitem.L_SHIPDATE >= date1) & (lineitem.L_SHIPDATE < date2)
    fpart = part[psel]
    fnation = nation[nsel]
    flineitem = lineitem[lsel]
    jn1 = fpart.merge(partsupp, left_on="P_PARTKEY", right_on="PS_PARTKEY")
    jn2 = jn1.merge(
        flineitem,
        left_on=["PS_PARTKEY", "PS_SUPPKEY"],
        right_on=["L_PARTKEY", "L_SUPPKEY"],
    )
    gb = jn2.groupby(["PS_PARTKEY", "PS_SUPPKEY", "PS_AVAILQTY"], as_index=False)[
        "L_QUANTITY"
    ].sum()
    gbsel = gb.PS_AVAILQTY > (0.5 * gb.L_QUANTITY)
    fgb = gb[gbsel]
    jn3 = fgb.merge(supplier, left_on="PS_SUPPKEY", right_on="S_SUPPKEY")
    jn4 = fnation.merge(jn3, left_on="N_NATIONKEY", right_on="S_NATIONKEY")
    jn4 = jn4.loc[:, ["S_NAME", "S_ADDRESS"]]
    total = jn4.sort_values("S_NAME").drop_duplicates()
    print(total.head())
    print("Q20 Execution time (s): ", time.time() - t1)


@bodo.jit
def q21(lineitem, orders, supplier, nation):
    t1 = time.time()
    lineitem_filtered = lineitem.loc[
                        :, ["L_ORDERKEY", "L_SUPPKEY", "L_RECEIPTDATE", "L_COMMITDATE"]
                        ]

    # Keep all rows that have another row in linetiem with the same orderkey and different suppkey
    lineitem_orderkeys = (
        lineitem_filtered.loc[:, ["L_ORDERKEY", "L_SUPPKEY"]]
        .groupby("L_ORDERKEY", as_index=False)["L_SUPPKEY"]
        .nunique()
    )
    lineitem_orderkeys.columns = ["L_ORDERKEY", "nunique_col"]
    lineitem_orderkeys = lineitem_orderkeys[lineitem_orderkeys["nunique_col"] > 1]
    lineitem_orderkeys = lineitem_orderkeys.loc[:, ["L_ORDERKEY"]]

    # Keep all rows that have l_receiptdate > l_commitdate
    lineitem_filtered = lineitem_filtered[
        lineitem_filtered["L_RECEIPTDATE"] > lineitem_filtered["L_COMMITDATE"]
        ]
    lineitem_filtered = lineitem_filtered.loc[:, ["L_ORDERKEY", "L_SUPPKEY"]]

    # Merge Filter + Exists
    lineitem_filtered = lineitem_filtered.merge(
        lineitem_orderkeys, on="L_ORDERKEY", how="inner"
    )

    # Not Exists: Check the exists condition isn't still satisfied on the output.
    lineitem_orderkeys = lineitem_filtered.groupby("L_ORDERKEY", as_index=False)[
        "L_SUPPKEY"
    ].nunique()
    lineitem_orderkeys.columns = ["L_ORDERKEY", "nunique_col"]
    lineitem_orderkeys = lineitem_orderkeys[lineitem_orderkeys["nunique_col"] == 1]
    lineitem_orderkeys = lineitem_orderkeys.loc[:, ["L_ORDERKEY"]]

    # Merge Filter + Not Exists
    lineitem_filtered = lineitem_filtered.merge(
        lineitem_orderkeys, on="L_ORDERKEY", how="inner"
    )

    orders_filtered = orders.loc[:, ["O_ORDERSTATUS", "O_ORDERKEY"]]
    orders_filtered = orders_filtered[orders_filtered["O_ORDERSTATUS"] == "F"]
    orders_filtered = orders_filtered.loc[:, ["O_ORDERKEY"]]
    total = lineitem_filtered.merge(
        orders_filtered, left_on="L_ORDERKEY", right_on="O_ORDERKEY", how="inner"
    )
    total = total.loc[:, ["L_SUPPKEY"]]

    supplier_filtered = supplier.loc[:, ["S_SUPPKEY", "S_NATIONKEY", "S_NAME"]]
    total = total.merge(
        supplier_filtered, left_on="L_SUPPKEY", right_on="S_SUPPKEY", how="inner"
    )
    total = total.loc[:, ["S_NATIONKEY", "S_NAME"]]
    nation_filtered = nation.loc[:, ["N_NAME", "N_NATIONKEY"]]
    nation_filtered = nation_filtered[nation_filtered["N_NAME"] == "SAUDI ARABIA"]
    total = total.merge(
        nation_filtered, left_on="S_NATIONKEY", right_on="N_NATIONKEY", how="inner"
    )
    total = total.loc[:, ["S_NAME"]]
    total = total.groupby("S_NAME", as_index=False).size()
    total.columns = ["S_NAME", "NUMWAIT"]
    total = total.sort_values(by=["NUMWAIT", "S_NAME"], ascending=[False, True])
    print(total.head())
    print("Q21 Execution time (s): ", time.time() - t1)


@bodo.jit
def q22(customer, orders):
    t1 = time.time()
    customer_filtered = customer.loc[:, ["C_ACCTBAL", "C_CUSTKEY"]]
    customer_filtered["CNTRYCODE"] = customer["C_PHONE"].str.slice(0, 2)
    customer_filtered = customer_filtered[
        (customer["C_ACCTBAL"] > 0.00)
        & customer_filtered["CNTRYCODE"].isin(
            ["13", "31", "23", "29", "30", "18", "17"]
        )
    ]
    avg_value = customer_filtered["C_ACCTBAL"].mean()
    customer_filtered = customer_filtered[customer_filtered["C_ACCTBAL"] > avg_value]
    # Select only the keys that don't match by performing a left join and only selecting columns with an na value
    orders_filtered = orders.loc[:, ["O_CUSTKEY"]].drop_duplicates()
    customer_keys = customer_filtered.loc[:, ["C_CUSTKEY"]].drop_duplicates()
    customer_selected = customer_keys.merge(
        orders_filtered, left_on="C_CUSTKEY", right_on="O_CUSTKEY", how="left"
    )
    customer_selected = customer_selected[customer_selected["O_CUSTKEY"].isna()]
    customer_selected = customer_selected.loc[:, ["C_CUSTKEY"]]
    customer_selected = customer_selected.merge(
        customer_filtered, on="C_CUSTKEY", how="inner"
    )
    customer_selected = customer_selected.loc[:, ["CNTRYCODE", "C_ACCTBAL"]]
    agg1 = customer_selected.groupby(["CNTRYCODE"], as_index=False).size()
    agg1.columns = ["CNTRYCODE", "NUMCUST"]
    agg2 = customer_selected.groupby(["CNTRYCODE"], as_index=False).agg(
        TOTACCTBAL=pd.NamedAgg(column="C_ACCTBAL", aggfunc="sum")
    )
    total = agg1.merge(agg2, on="CNTRYCODE", how="inner")
    total = total.sort_values(by=["CNTRYCODE"], ascending=[True])
    print(total)
    print("Q22 Execution time (s): ", time.time() - t1)



In [16]:
run_queries(iceberg_url)

%px:   0%|          | 0/72 [00:00<?, ?tasks/s]

[stdout:0] ################################################################################################################################
Hello
################################################################################################################################
  L_RETURNFLAG L_LINESTATUS  L_QUANTITY  L_EXTENDEDPRICE    DISC_PRICE  \
1            A            F  37734107.0     5.658655e+10  5.375826e+10   
2            N            F    991417.0     1.487505e+09  1.413082e+09   
3            N            O  74476040.0     1.117017e+11  1.061182e+11   
0            R            F  37719753.0     5.656804e+10  5.374129e+10   

         CHARGE    AVG_QTY     AVG_PRICE  L_DISCOUNT  L_ORDERKEY  
1  5.590907e+10  25.522006  38273.129735    0.049985     1478493  
2  1.469649e+09  25.516472  38284.467761    0.050093       38854  
3  1.103670e+11  25.502227  38249.117989    0.049997     2920374  
0  5.588962e+10  25.505794  38250.854626    0.050009     1478870  
Q01 Execution time 

[stderr:0] /opt/conda/lib/python3.10/site-packages/bodo/io/parquet_pio.py:861: BodoWarning: Total number of row groups in parquet dataset ['s3://bodo-iceberg-test2/arctic_test/SF1_LINEITEM_PQ_A/data/00000-1-96b67062-b249-4e91-a139-37ecf0d34015-00001.parquet', 's3://bodo-iceberg-test2/arctic_test/SF1_LINEITEM_PQ_A/data/00001-2-b449f2ba-5736-4b48-a705-bc54b6790907-00001.parquet', 's3://bodo-iceberg-test2/arctic_test/SF1_LINEITEM_PQ_A/data/00002-3-4b893077-86f3-47df-b6a5-2e44e1088da1-00001.parquet', 's3://bodo-iceberg-test2/arctic_test/SF1_LINEITEM_PQ_A/data/00003-4-99de0a1d-39c5-43ee-a3d5-fc5082fbd960-00001.parquet', 's3://bodo-iceberg-test2/arctic_test/SF1_LINEITEM_PQ_A/data/00004-5-d52eeda9-1593-4978-b486-c20b16206692-00001.parquet', 's3://bodo-iceberg-test2/arctic_test/SF1_LINEITEM_PQ_A/data/00005-6-763d3a9c-9108-4736-8db5-978616c328d7-00001.parquet', 's3://bodo-iceberg-test2/arctic_test/SF1_LINEITEM_PQ_A/data/00006-7-0f4bff4d-687e-4371-beb3-b9929b3feba4-00001.parquet', 's3://bodo-ice

[stdout:67] Empty DataFrame
Columns: [N_NAME, TMP]
Index: []
       PS_PARTKEY       VALUE
20230        8114  7933921.88
4586        71518  7932261.69
1516        72922  7930400.64
5248       146699  7929167.40
16107       92387  7928972.67
25294      186289  7928786.19
20137       95952  7927972.78
4091       196514  7927180.70
1720         4403  7925729.04
17901        2267  7925649.37
6751        45924  7925047.68
15632       11493  7916722.23
21106      104478  7916253.60
7590       166794  7913842.00
23338      161995  7910874.27
Empty DataFrame
Columns: [CNTRYCODE, NUMCUST, TOTACCTBAL]
Index: []
Execution Completed:..


[stdout:62] Empty DataFrame
Columns: [N_NAME, TMP]
Index: []
       PS_PARTKEY       VALUE
24423      121622  8018462.17
20430       14357  8016906.30
22144      135601  8016209.44
10649       58458  8016192.52
29209       73036  8015799.00
15358      184722  8015680.31
27034      151664  8014821.96
13194      195090  8012680.20
21442      162609  8011241.00
17403       83532  8009753.85
14457       50166  8007137.89
11221      181562  8006805.96
20748      175165  8005319.76
15425       62500  8005316.28
Empty DataFrame
Columns: [CNTRYCODE, NUMCUST, TOTACCTBAL]
Index: []
Execution Completed:..


...

[stdout:31] Empty DataFrame
Columns: [N_NAME, TMP]
Index: []
       PS_PARTKEY       VALUE
4750        97744  8785315.94
26381      164327  8784503.86
17647       76542  8782613.28
2565         4731  8772846.70
19332      157590  8772006.45
16831      154276  8771733.91
9467        28705  8771576.64
15601      100226  8769455.00
8392       179195  8769185.16
8696       184355  8768118.05
19517      120408  8768011.12
16235       63145  8761991.96
20463       53135  8753491.80
6759       173071  8750508.80
Empty DataFrame
Columns: [CNTRYCODE, NUMCUST, TOTACCTBAL]
Index: []
Execution Completed:..


[stdout:59] Empty DataFrame
Columns: [N_NAME, TMP]
Index: []
       PS_PARTKEY       VALUE
20643       16167  8067495.24
22370       12662  8067248.85
15653       94540  8063727.16
18646       23308  8063463.18
26305       27390  8062823.25
29753      130660  8062787.48
1219         8608  8062411.16
9527       181552  8062008.30
3329       199319  8060248.56
6553        55475  8058850.92
24810      142711  8057926.58
8843       103499  8056978.00
819        105943  8056698.75
3694         8432  8053052.16
Empty DataFrame
Columns: [CNTRYCODE, NUMCUST, TOTACCTBAL]
Index: []
Execution Completed:..


[stdout:21] Empty DataFrame
Columns: [N_NAME, TMP]
Index: []
       PS_PARTKEY       VALUE
15122      125329  9198688.50
17159      167867  9194022.72
14794      134767  9191444.72
1871       127745  9191271.56
4220        69208  9187110.00
3094       155222  9186469.16
15743      196916  9182995.82
5322       195590  9176353.12
291        169155  9175176.09
2845        81558  9171946.50
14134      185136  9171293.04
27808      114790  9168509.10
5730       194142  9165836.61
27856      167639  9161165.00
27556       11241  9160789.46
Empty DataFrame
Columns: [CNTRYCODE, NUMCUST, TOTACCTBAL]
Index: []
Execution Completed:..


[stdout:71] Empty DataFrame
Columns: [N_NAME, TMP]
Index: []
   O_YEAR  MKT_SHARE
1    1995   0.034436
0    1996   0.041486
Empty DataFrame
Columns: [PS_PARTKEY, VALUE]
Index: []
Empty DataFrame
Columns: [CNTRYCODE, NUMCUST, TOTACCTBAL]
Index: []
Execution Completed:..


[stdout:7] Empty DataFrame
Columns: [N_NAME, TMP]
Index: []
       PS_PARTKEY        VALUE
26282      156391  10839810.38
27663       40884  10837234.19
10778      141288  10837130.21
1421       152388  10830977.82
7223        33449  10830858.72
13592      149035  10826130.02
29660      162620  10814275.68
7326       118324  10791788.10
8855        38932  10777541.75
10400      121294  10764225.22
12130       48721  10762582.49
4214        63342  10740132.60
14211        5614  10724668.80
7932        62266  10711143.10
Empty DataFrame
Columns: [CNTRYCODE, NUMCUST, TOTACCTBAL]
Index: []
Execution Completed:..
